# Assignment 1: Dynamic Programming

---

## Task 1) Edit Distances

Implement the [Hamming](https://en.wikipedia.org/wiki/Hamming_distance) and [Levenshtein](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit) distances and compute them for the for the following word pairs.
It may help to compute them by hand first.

<img src = "./assets/97090.jpg" width="33%" /> <img src = "./assets/97222.jpg" width="33%" /> <img src = "./assets/97669.jpg" width="33%" />

Aside from computing the distance (which is a scalar), do the backtrace and compute the edit transcript (and thus alignment).

---

In [1]:
WORD_PAIRS = [
    ("GCGTATGAGGCTAACGC", "GCTATGCGGCTATACGC"),
    ("kühler schrank", "schüler krank"),
    ("the longest", "longest day"),
    ("nicht ausgeloggt", "licht ausgenockt"),
    ("gurken schaben", "schurkengaben")
]

In [2]:
def hamming(s1: str, s2: str) -> int:
    """
    Compute the hamming distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int.
    """
    # Hint: Think about how you can deal with unequal word lengths.
    
    ### YOUR CODE HERE

    if len(s2) < len(s1):
        s1, s2 = s2, s1
    return sum(int(s2[i] != c) for i, c in enumerate(s1)) + len(s2) - len(s1)
    
    ### END YOUR CODE

In [3]:
for wordpair in WORD_PAIRS:
    dist = hamming(s1=wordpair[0], s2=wordpair[1])
    print("hamming('{}', '{}') = {}".format(
        wordpair[0], wordpair[1], dist
    ))

### EXPECTED
# hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
# hamming('kühler schrank', 'schüler krank') = 13
# hamming('the longest', 'longest day') = 11
# hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
# hamming('gurken schaben', 'schurkengaben') = 14

hamming('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 10
hamming('kühler schrank', 'schüler krank') = 13
hamming('the longest', 'longest day') = 11
hamming('nicht ausgeloggt', 'licht ausgenockt') = 4
hamming('gurken schaben', 'schurkengaben') = 14


In [4]:
import numpy as np

def levenshtein(s1: str, s2: str, cost={'m': 0, 's': 1, 'i': 1, 'd': 1}) -> tuple[int, str]:
    """
    Compute the levenshtein (edit) distance between two strings.

    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.

    Returns the distance as int and edit transcript as string.
    """
    # Hint: The probably most intuitive approach is bottom-up.
    
    ### YOUR CODE HERE

    D = np.zeros((len(s1) + 1, len(s2) + 1), dtype=np.int32)
    paths = np.ndarray((len(s1), len(s2)), dtype=np.byte)
    D[1:, 0] = np.arange(1, len(s1) + 1)
    D[0, 1:] = np.arange(1, len(s2) + 1)
    for i, c1 in enumerate(s1):
        for j, c2 in enumerate(s2):
            cd = D[i, j+1] + cost["d"]
            ci = D[i+1, j] + cost["i"]
            D[i+1, j+1] = D[i, j]
            if c1 != c2:
                D[i+1, j+1] += cost["s"]
                op = "s"
            else:
                D[i+1, j+1] += cost["m"]
                op = "m"
            if cd < D[i+1, j+1]:
                D[i+1, j+1] = cd
                op = "d"
            if ci <= D[i+1, j+1]:
                D[i+1, j+1] = ci
                op = "i"
            paths[i, j] = ord(op)
    i = len(s1) - 1
    j = len(s2) - 1
    path = ""
    while i != -1 and j != -1:
        op = chr(paths[i, j])
        path = op + path
        if op != "i":
            i -= 1
        if op != "d":
            j -= 1
    path = "d" * (i + 1) + "i" * (j + 1) + path
    return D[-1, -1], path
    ### END YOUR CODE

In [5]:
for wordpair in WORD_PAIRS:
    dist, transcript = levenshtein(s1=wordpair[0], s2=wordpair[1])
    print("levenshtein('{}', '{}') = {} ({})".format(
        wordpair[0], wordpair[1], dist, transcript
    ))

### EXPECTED
# levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
# levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmsddmmmm)
# levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
# levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
# levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmsdddmmmm)

levenshtein('GCGTATGAGGCTAACGC', 'GCTATGCGGCTATACGC') = 3 (mmdmmmmsmmmmmimmmm)
levenshtein('kühler schrank', 'schüler krank') = 6 (ssmimmmmddsmmmm)
levenshtein('the longest', 'longest day') = 8 (ddddmmmmmmmiiii)
levenshtein('nicht ausgeloggt', 'licht ausgenockt') = 4 (smmmmmmmmmmsmssm)
levenshtein('gurken schaben', 'schurkengaben') = 7 (siimmmmmdddsmmmm)


---

## Task 2) Basic Spelling Correction using Edit Distance

For spelling correction, we will use prior knowledge, to put _some_ learning into our system.

The underlying idea is the _Noisy Channel Model_, that is: The user _intends_ to write a word `w`, but through some noise in the process, happens to type the word `x`.

The correct word $\hat{w}$ is that word, that is a valid candidate and has the highest probability:

$$
\begin{eqnarray}
\DeclareMathOperator*{\argmax}{argmax}
\hat{w} & = & \argmax_{w \in V} P(w | x) \\
        & = & \argmax_{w \in V} \frac{P(x|w) P(w)}{P(x)} \\
        & = & \argmax_{w \in V} P(x|w) P(w)
\end{eqnarray}
$$

1. The candidates $V$ can be obtained from a _vocabulary_.
2. The probability $P(w)$ of a word $w$ can be _learned (counted) from data_.
3. The probability $P(x|w)$ is more complicated... It could be learned from data, but we could also use a _heuristic_ that relates to the edit distance, e.g. rank by distance.

You may use [Peter Norvig's count_1w.txt](http://norvig.com/ngrams/) file, [local mirror](res/count_1w.tar.bz2).
Note that it may help to restrict to the first 10k words to get started.

---

In [6]:
EXAMPLES = [
    "pirates",    # in-voc
    "pirutes",    # pirates?
    "continoisly",  # continuosly?
]

In [7]:
import requests
import re
import os

### TODO: Prepare the vocabulary

### YOUR CODE HERE

WORDS_URL = "http://norvig.com/ngrams/count_1w.txt"
WORDS_FILE_PATH = "data/count_1w.txt"

if not os.path.exists(WORDS_FILE_PATH):
    text = requests.get(WORDS_URL).text
    with open(WORDS_FILE_PATH, "w") as fp:
        fp.write(text)
else:
    with open(WORDS_FILE_PATH) as fp:
        text = fp.read()

V = []
counts = []
total = 0
for line in text.split("\n"):
    match = re.match(r"^(\w+)\s+(\d+)$", line)
    if match is not None:
        V.append(match.group(1))
        count = int(match.group(2))
        counts.append(count)
        total += count
P = { w: counts[i] / total for i, w in enumerate(V) }

### END YOUR CODE

In [8]:
from functools import cache
import math
from concurrent.futures import ProcessPoolExecutor
from typing import Callable, Sequence

# assumption: typos are poisson distributed with 1 typo per 200 characters on average

@cache
def fac(x: int) -> int:
    assert x >= 0
    if x < 2:
        return 1
    return fac(x - 1) * x

# Stirling's formula for factorial approximateion
def fac_approx(x: int) -> float:
    if x <= 20:
        return fac(x)
    return math.sqrt(2 * math.pi * x) * (x / math.e) ** x

def poisson(k: int, l: float) -> float:
    return l ** k / (fac_approx(k) * math.e ** l)

TYPO_PER_CHAR = 0.005
MAX_TYPOS_PER_CHAR = 0.5
BATCH_SIZE = 256

def _dists_probs(args: tuple[Callable[[str, str], tuple[int, str]], str, str, float]) -> tuple[int, float]:
    dist_fn, w, w_, prior = args
    dist = dist_fn(w, w_)[0]
    if dist >= MAX_TYPOS_PER_CHAR * len(w):
        return dist, 0
    prob = prior * poisson(dist, TYPO_PER_CHAR)
    return dist, prob

def suggest(w: str, dist_fn, max_cand=5) -> Sequence[tuple[str, int, float]]:
    """
    Compute suggestions for spelling correction using edit distance.
    
    Arguments:
    w: Word in question.
    dist_fn: Distance function to use (e.g. levenshtein).
    max_cand: Maximum number of suggestions.

    Returns a list of tuples (word, dist, score) sorted by score and distance.
    """
    ### YOUR CODE HERE
    
    with ProcessPoolExecutor() as exec:
        dists_probs_ll = []
        # batched processing required due to limitations of ProcessPoolExecutor
        for i in range(math.ceil(len(V) / BATCH_SIZE)):
            k = i * BATCH_SIZE
            dists_probs_ll.append(list(exec.map(
                _dists_probs,
                ((dist_fn, w_, w, P[w_]) for w_ in V[k:k+BATCH_SIZE])
            )))
    candidates: list[tuple[str, int, float]] = [("", 0, 0) for _ in range(max_cand)]
    for i, (dist, prob) in enumerate((x for l in dists_probs_ll for x in l)):
        j = max_cand - 1
        if candidates[j][2] < prob:
            while j > 0 and candidates[j - 1][2] < prob:
                candidates[j] = candidates[j-1]
                j -= 1
            w_ = V[i]
            candidates[j] = w_, dist, prob
    i = max_cand
    while i > 0 and candidates[i - 1][2] == 0:
        i -= 1
    return candidates[:i]

    
    ### END YOUR CODE

In [9]:
# How does your suggest function behave with levenshtein distance?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=levenshtein, max_cand=3)
    print("{} {}".format(word, suggestions))

### EXPECTED
### Notice: your scores may vary!
# pirates [('pirates', 0, -11.408058827802126)]
# pirutes [('pirates', 1, -11.408058827802126), ('minutes', 2, -8.717825438953103), ('viruses', 2, -11.111468702571859)]
# continoisly [('continously', 1, -15.735337826575178), ('continuously', 2, -11.560071979871001), ('continuosly', 2, -17.009283000138204)]

pirates [('pirates', 0, 1.105023539179934e-05), ('pirate', 1, 3.644597024494112e-08), ('pilates', 1, 2.0433874060751604e-08)]
pirutes [('pirates', 1, 5.52511769589967e-08), ('minutes', 2, 2.0353310511005172e-09), ('viruses', 2, 1.8581852022600007e-10)]
continoisly [('continously', 1, 7.295047817732711e-10), ('continuously', 2, 1.1864872327588812e-10), ('continuosly', 2, 5.101534083210598e-13)]


### Food for Thought

- How would you modify the implementation so that it works fast for large lexica (eg. the full file above)?
- How would you modify the implementation so that it works "while typing" instead of at the end of a word?
- How do you handle unknown/new words?

---

## Task 3) Needleman-Wunsch: Keyboard-aware Auto-Correct

In the parts 1 and 2 above, we applied uniform cost to all substitutions.
This does not really make sense if you look at a keyboard: the QWERTY layout will favor certain substitutions (eg. _a_ and _s_), while others are fairly unlikely (eg. _a_ and _k_).

Implement the [Needleman-Wunsch algorithm](https://en.wikipedia.org/wiki/Needleman–Wunsch_algorithm) which is very similar to the [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance), however it doesn't _minimize the cost_ but _maximizes the similarity_.
For a good measure of similarity, implement a metric that computes a meaningful weight for a given character substitution.

---

In [12]:
from numpy import linalg as la

KEY_COORDINATES = {
    "q": np.array([0, 0]),
    "w": np.array([1, 0]),
    "e": np.array([2, 0]),
    "r": np.array([3, 0]),
    "t": np.array([4, 0]),
    "y": np.array([5, 0]),
    "u": np.array([6, 0]),
    "i": np.array([7, 0]),
    "o": np.array([8, 0]),
    "p": np.array([9, 0]),
    "a": np.array([0.33, 1]),
    "s": np.array([1.33, 1]),
    "d": np.array([2.33, 1]),
    "f": np.array([3.33, 1]),
    "g": np.array([4.33, 1]),
    "h": np.array([5.33, 1]),
    "j": np.array([6.33, 1]),
    "k": np.array([7.33, 1]),
    "l": np.array([8.33, 1]),
    "z": np.array([0.67, 2]),
    "x": np.array([1.67, 2]),
    "c": np.array([2.67, 2]),
    "v": np.array([3.67, 2]),
    "b": np.array([4.67, 2]),
    "n": np.array([5.67, 2]),
    "m": np.array([6.67, 2])    
}

def keyboardsim(s1: str, s2: str) -> float:
    ### YOUR CODE HERE

    s1 = s1.lower()
    s2 = s2.lower()
    assert s1 in KEY_COORDINATES
    assert s2 in KEY_COORDINATES
    return (1 / (1 + la.norm(KEY_COORDINATES[s2] - KEY_COORDINATES[s1]))).item()
    
    ### END YOUR CODE


def nw(s1: str, s2: str, d: float, sim_fn) -> float:
    """
    Apply needleman-wunsch algorithm.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    d: Gap penalty score.
    sim_fn: Similarity function to use.

    Returns the score as float.
    """
    ### YOUR CODE HERE
    
    D = np.zeros((len(s1) + 1, len(s2) + 1))
    D[1:, 0] = np.arange(1, len(s1) + 1) * d
    D[0, 1:] = np.arange(1, len(s2) + 1) * d
    for i, c1 in enumerate(s1):
        for j, c2 in enumerate(s2):
            D[i+1, j+1] = max(
                D[i, j] + keyboardsim(c1, c2),
                D[i, j+1] + d,
                D[i+1, j] + d
            )
    return D[-1, -1]
    
    ### END YOUR CODE


def nw_based_dist(s1: str, s2: str) -> tuple[int, str]:
    """
    Compute the needleman-wunsch distance between two strings.
    
    Arguments:
    s1: First string of word pair.
    s2: Second string of word pair.
    
    Returns the distance as int and <unsupported> string.
    """
    ### YOUR CODE HERE
    
    return round(len(s1) - nw(s1, s2, -0.5, keyboardsim)), "<unsuppoorted>"
    
    ### END YOUR CODE

In [13]:
# How does your suggest function behave with nw and a keyboard-aware similarity?

for word in EXAMPLES:
    suggestions = suggest(w=word, dist_fn=nw_based_dist, max_cand=3)
    print("{} {}".format(word, suggestions))

pirates [('pirates', 0, 1.105023539179934e-05), ('pirate', 0, 7.289194048988224e-06), ('rates', 1, 1.1596682843072494e-06)]
pirutes [('pirates', 1, 5.52511769589967e-08), ('pirate', 1, 3.644597024494112e-08), ('price', 2, 1.0608924376418283e-08)]
continoisly [('continously', 0, 1.4590095635465423e-07), ('continuing', 2, 5.377825200587693e-10), ('continues', 2, 5.076294992502114e-10)]


### Food for Thought

- What could be better heuristics for similarity resp. cost of substitution than the one above?
- What about capitalization, punctiation and special characters?
- What about swipe-to-type?